In [1]:
import math

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from IPython.display import display
from sklearn.neighbors import KernelDensity
# %matplotlib ipympl
%matplotlib qt5
# %matplotlib widget
# matplotlib.use("nbagg")  # interactive !
# matplotlib.use('Qt5Agg')

In [2]:
def getExtremePoints(data, typeOfInflexion=None, maxPoints=None):
    """
    This method returns the indeces where there is a change in the trend of the input series.
    typeOfInflexion = None returns all inflexion points, max only maximum values and min
    only min,
    """
    a = np.diff(data)
    asign = np.sign(a)
    signchange = ((np.roll(asign, 1) - asign) != 0).astype(int)
    idx = np.where(signchange == 1)[0]

    if typeOfInflexion == 'max' and data[idx[0]] < data[idx[1]]:
        idx = idx[1:][::2]

    elif typeOfInflexion == 'min' and data[idx[0]] > data[idx[1]]:
        idx = idx[1:][::2]
    elif typeOfInflexion is not None:
        idx = idx[::2]

    # sort ids by min value
    if 0 in idx:
        idx = np.delete(idx, 0)
    if (len(data) - 1) in idx:
        idx = np.delete(idx, len(data) - 1)
    idx = idx[np.argsort(data[idx])]
    # If we have maxpoints we want to make sure the timeseries has a cutpoint
    # in each segment, not all on a small interval
    if maxPoints is not None:
        idx = idx[:maxPoints]
        if len(idx) < maxPoints:
            return (np.arange(maxPoints) + 1) * (len(data) // (maxPoints + 1))

    return idx


from scipy import ndimage as ndi


def local_maxima_3D(data, order=1):
    """Detects local maxima in a 3D array

    Parameters
    ---------
    data : 3d ndarray
    order : int
        How many points on each side to use for the comparison

    Returns
    -------
    coords : ndarray
        coordinates of the local maxima
    values : ndarray
        values of the local maxima
    """
    size = 1 + 2 * order
    footprint = np.ones((size, size, size))
    footprint[order, order, order] = 0

    filtered = ndi.maximum_filter(data, footprint=footprint)
    mask_local_maxima = data > filtered
    coords = np.asarray(np.where(mask_local_maxima)).T
    values = data[mask_local_maxima]

    return coords, values

In [3]:
# i = 767
# i = 877
# i = 987 # abnormal?
# i = 452
# i = 356 # good shape
# i = 333 # good
# i = 803 # good not enough
# i = 111 # looks good, using this
# i = 85
# i = 86 # strange
# i = 91  # edge density
# i = 106  # should use this
# i = 95 # classic
# i = 99 # classic
# i = 206
# i = 213
# i = 224
# i = 244
# i = 403
# i = 405
# i = 408
i = 413
trj = np.load(f'../data/SHL_msk3_features/clean_trj_segs.npy', allow_pickle=True)[i]
fs = np.load(f'../data/SHL_msk3_features/clean_multi_feature_segs.npy', allow_pickle=True)[i]
fs_msk = np.load(f'../data/SHL_msk3_features/fs_seg_masks.npy', allow_pickle=True)[i]

d = fs[2]
v = fs[3]
a = fs[4]
jk = fs[5]
hc = fs[7]
hcr = fs[8]

x = trj[0]
y = trj[1]
x_ori = np.copy(x)
y_ori = np.copy(y)
d_ = np.vstack([x, y]).T

minx, miny = min(x), min(y)

n = len(x)



In [4]:
plt.figure()
plt.plot(d)
plt.plot(v)
plt.plot(a)
plt.plot(jk)
plt.plot(hc)
plt.show()

In [5]:
# plt.figure(figsize=(15,15))
plt.figure()
# plt.title('trj')
# plt.scatter(x, y)
plt.plot(x, y, color='red', marker='o', markerfacecolor='white',
         markeredgecolor='#1f77b4', markeredgewidth=1.5)
plt.xlabel("Latitude", fontweight='bold')
plt.ylabel("Longitude", fontweight='bold')
plt.ticklabel_format(style='plain', axis='x', useOffset=False)
plt.xticks(rotation=30)
plt.show()

#### scale!

In [6]:
from sklearn.preprocessing import MinMaxScaler

target_scale = 100  # 0~100
grid_size = 100

# transform trj to same scale as the grid of kde
scaler = MinMaxScaler(feature_range=(0, 1))
x_s = scaler.fit_transform(x.reshape(-1, 1)).squeeze() * target_scale
y_s = scaler.fit_transform(y.reshape(-1, 1)).squeeze() * target_scale
plt.figure()
plt.title('trj scaled')

plt.plot(x_s, y_s, color='red', marker='o', markerfacecolor='white',
         markeredgecolor='#1f77b4', markeredgewidth=1.5)
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.show()

#### Create meshgrid

In [7]:
deltaX = (max(x) - min(x)) / 10
deltaY = (max(y) - min(y)) / 10

xmin = min(x) - deltaX
xmax = max(x) + deltaX

ymin = min(y) - deltaY
ymax = max(y) + deltaY

# Create meshgrid
x_mg, y_mg = np.mgrid[xmin:xmax:complex(0, grid_size), ymin:ymax:complex(0, grid_size)]


#### Boundary correction using mirroring, then do kde
https://kdepy.readthedocs.io/en/latest/examples.html#boundary-correction-using-mirroring

In [8]:
data = np.vstack([x_s, y_s]).T  # use scaled data!!!.

# n_bdry_ext = round(n * .1)
# begin_ext = data[1:n_bdry_ext][::-1] # note: remove 1st repeat element
# end_ext = data[-n_bdry_ext:-1][::-1] #
# data_ext = np.vstack([begin_ext, data, end_ext])

# x_s_mir = np.concatenate([(2 * x_s.min() - x_s)[:-1], x_s, (2 * x_s.max() - x_s)[1:]])
# y_s_mir = np.concatenate([(2 * y_s.min() - y_s)[:-1], y_s, (2 * y_s.max() - y_s)[1:]])
x_s_mir = np.concatenate([(2 * x_s.min() - x_s), x_s, (2 * x_s.max() - x_s)])
y_s_mir = np.concatenate([(2 * y_s.min() - y_s), y_s, (2 * y_s.max() - y_s)])
data_mir = np.vstack([x_s_mir, y_s_mir]).T  # use scaled data!!!.
data_mir.shape  # n points and each point dim

(384, 2)

mirrored, hence the grid_size to do kde is 3 times

In [9]:
grid_size_mir = grid_size * 3

In [10]:
plt.figure()
plt.title('trj scaled mirrored')

plt.plot(x_s_mir, y_s_mir, color='red', marker='o', markerfacecolor='white',
         markeredgecolor='#1f77b4', markeredgewidth=1.5)
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.show()

## FFTKDE

#### fix grid error:
because i found that if we do not manually generate grid using np.linspace, the peak values in kde detected is lagged!!

https://github.com/tommyod/KDEpy/issues/15

In [11]:
from KDEpy import FFTKDE

# Create 2D grid
kde_grid_x = np.linspace(data_mir.min() - 1, data_mir.max() + 1, grid_size_mir)  # "-1, +1" is used to ensure range
kde_grid_y = np.linspace(data_mir.min() - 1, data_mir.max() + 1, grid_size_mir)
kde_grid = np.stack(np.meshgrid(kde_grid_x, kde_grid_y), -1).reshape(-1, 2)
kde_grid[:, [0, 1]] = kde_grid[:, [1, 0]]  # Swap indices

In [12]:
from KDEpy import TreeKDE
print()
fit = FFTKDE(bw=1, kernel='epa').fit(data_mir)
z_kde = fit.evaluate(kde_grid)
z_kde_grid = z_kde.reshape(grid_size_mir, grid_size_mir).T


In [13]:
# xff_mir, yff_mir = np.unique(xy_mir[:, 0]), np.unique(xy_mir[:, 1])
# zffr_mir = z_kde_mir.reshape(grid_size_mir, grid_size_mir).T


In [14]:

plt.figure()
N = 8  # Number of contours
plt.title('contours of mirrored kde')
plt.contour(kde_grid_x, kde_grid_y, z_kde_grid, N, linewidths=0.8, colors="k")
plt.contourf(kde_grid_x, kde_grid_y, z_kde_grid, N, cmap="PuBu")
plt.plot(data[:, 0], data[:, 1], "ok", ms=2)
# plt.gca().invert_yaxis()
# plt.yticks([])
# plt.xticks([])
plt.show()

### MIDDLE PART (NON MIRROR)

#### take out middle part (non mirror)

In [15]:
kde_grid_x_mid = kde_grid_x[grid_size:2 * grid_size]
kde_grid_y_mid = kde_grid_y[grid_size:2 * grid_size]
z_kde_grid_mid = z_kde_grid[grid_size:2 * grid_size, grid_size:2 * grid_size]

#### 3d kde of middle part (non mirror)

In [29]:
fig = plt.figure()
ax = plt.axes(projection='3d')
# surf = ax.plot_surface(xff, yff, zffr, rstride=1, cstride=1, cmap='coolwarm', edgecolor='none')
surf = ax.plot_surface(x_mg, y_mg, z_kde_grid_mid, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
ax.set_xlabel('Latitude', labelpad=18, fontweight='bold')
ax.set_ylabel('Longitude', labelpad=8, fontweight='bold')
ax.set_zlabel('PDF', labelpad=8, fontweight='bold')
# ax.set_title(' 3d kde of middle part (non mirror)')
# ax.set_ylim3d( y_mg.max(), y_mg.min(),)
ax.set_xlim3d(x_mg.max(), x_mg.min(), )  # to make the axis values order same as 2d plot
ax.ticklabel_format(useOffset=False)
ax.tick_params(axis='x', which='major', pad=8)
ax.tick_params(axis='y', which='major', pad=3)
fig.colorbar(surf, shrink=.3, aspect=5)  # add color bar indicating the PDF
ax.view_init(60, 35)
# plt.show()

#### peak_local_max of middle part

In [17]:

from skimage.feature import peak_local_max

pk_coords_mid = peak_local_max(z_kde_grid_mid, exclude_border=False, threshold_rel=0.3,
                               min_distance=round(
                                   grid_size * .05))  # coordinate!!  note the ration .3 and min_dsitance 2

#### show middle part kde image

In [18]:
plt.figure()
# plt.title('z_kde_grid_mid')
plt.imshow(z_kde_grid_mid, interpolation='none', cmap='viridis')
plt.plot(pk_coords_mid[:, 1], pk_coords_mid[:, 0], 'r.')  # note columns' order
# plt.plot(pk_coords_mid[:, 1], pk_coords_mid[:, 0], color='grey', marker='.')  # note columns' order
plt.xlabel("Latitude", fontweight='bold')
plt.ylabel("Longitude", fontweight='bold')
plt.colorbar()
ax=plt.gca()
ax.invert_yaxis()
x_ticks = ['', '50.9625', '50.9626', '50.9627', '50.9628', '50.9629', '50.9630' , '50.9631']
y_ticks = ['' ,'0.0960', '0.0965', '0.0970', '0.0975']
plt.xticks(np.arange(0,100, 100/8),  x_ticks, rotation=30)
plt.yticks(np.arange(0,100, 100/5), y_ticks)
# ax.set_xticklabels(['', '50.9625', '50.9626', '50.9627', '50.9628', '50.9629', '50.9630' , '50.9631'])
# ax.set_yticklabels(['' ,'0.0960', '0.0965', '0.0970', '0.0975'])
plt.show()

### ENTIRE PART

#### do  peaklocalmax for entire mirrored result

In [19]:
from skimage.feature import peak_local_max

# coordinate!!  note the ratio and distance
pk_coords_mir = peak_local_max(z_kde_grid, exclude_border=False, threshold_rel=0.1,
                               min_distance=2)
# pk_coords_mir = peak_local_max(z_kde_grid, exclude_border=False,)
pk_coords_mir

array([[ 79,  33],
       [121, 167],
       [277, 231],
       [271, 238],
       [127, 160],
       [ 73,  40],
       [238, 290],
       [160, 108],
       [ 40,  92],
       [242, 284],
       [156, 114],
       [ 44,  86],
       [ 28,  99],
       [172, 101],
       [226, 297],
       [262, 247],
       [136, 151],
       [ 64,  49],
       [ 55,  61],
       [145, 139],
       [253, 259],
       [ 16,  87],
       [184, 113],
       [214, 285],
       [231, 293],
       [167, 105],
       [ 33,  95],
       [251, 267],
       [147, 131],
       [  9,  78],
       [191, 122],
       [ 53,  69],
       [207, 276],
       [285, 223],
       [113, 175],
       [ 87,  25],
       [ 97,   8],
       [103, 192],
       [295, 206],
       [294, 210],
       [104, 188],
       [ 96,  12],
       [ 68,  45],
       [132, 155],
       [266, 243],
       [ 99,   2],
       [101, 198],
       [297, 200],
       [220, 291],
       [178, 107],
       [ 22,  93],
       [ 50,  74],
       [150,

In [20]:
def find_peak_repeatedly(data, min_peaks=3 * 4, max_peaks=3 * 9, threshold_rel=0, min_distance=2):
    """
    To avoid the situation that a very high peak occurred lead to the rest peaks cannot be identified,
    hence we repeatedly find peaks by making already identified peaks equal to zero until the `min_peaks` is met.
    Besides, the `max_peaks` is also required, if peaks more than `max_peaks` is detected, we only need the former highest
    `max_peaks`.

    Returns: indices of peaks

    """
    data_cp = np.copy(data)

    results = None
    n_pks = 0
    while n_pks <= min_peaks:
        pks = peak_local_max(data_cp, exclude_border=False, threshold_rel=threshold_rel, min_distance=min_distance)
        print(n_pks, pks)
        if results is None:
            results = pks
        else:
            results = np.concatenate([results, pks])
        n_pks += len(pks)
        # results += pks
        for p in pks:
            data_cp[p[0], p[1]] = 0

    if n_pks > max_peaks:
        # print([coord for coord in results])
        pk_vals = np.array([[*coord, data[coord[0], coord[1]]] for coord in results])
        # sort by peak val
        pk_vals = pk_vals[pk_vals[:, 2].argsort()][::-1]
        results = pk_vals[:max_peaks, [0, 1]].astype(int)
    return np.array(results)


print()
pk_coords_mir = find_peak_repeatedly(z_kde_grid, min_peaks=3 * 4, max_peaks=3 * 16, threshold_rel=0.1, min_distance=2)
print()
pk_coords_mir


0 [[ 79  33]
 [121 167]
 [277 231]
 [271 238]
 [127 160]
 [ 73  40]
 [238 290]
 [160 108]
 [ 40  92]
 [242 284]
 [156 114]
 [ 44  86]
 [ 28  99]
 [172 101]
 [226 297]
 [262 247]
 [136 151]
 [ 64  49]
 [ 55  61]
 [145 139]
 [253 259]
 [ 16  87]
 [184 113]
 [214 285]
 [231 293]
 [167 105]
 [ 33  95]
 [251 267]
 [147 131]
 [  9  78]
 [191 122]
 [ 53  69]
 [207 276]
 [285 223]
 [113 175]
 [ 87  25]
 [ 97   8]
 [103 192]
 [295 206]
 [294 210]
 [104 188]
 [ 96  12]
 [ 68  45]
 [132 155]
 [266 243]
 [ 99   2]
 [101 198]
 [297 200]
 [220 291]
 [178 107]
 [ 22  93]
 [ 50  74]
 [150 126]
 [248 272]
 [291 217]
 [107 181]
 [ 93  19]
 [ 58  55]
 [257 253]
 [142 144]
 [  1  66]
 [199 134]
 [199 264]
 [  3  70]
 [197 130]
 [201 268]]



array([[ 79,  33],
       [121, 167],
       [277, 231],
       [271, 238],
       [127, 160],
       [ 73,  40],
       [238, 290],
       [160, 108],
       [ 40,  92],
       [242, 284],
       [156, 114],
       [ 44,  86],
       [ 28,  99],
       [172, 101],
       [226, 297],
       [262, 247],
       [136, 151],
       [ 64,  49],
       [ 55,  61],
       [145, 139],
       [253, 259],
       [ 16,  87],
       [184, 113],
       [214, 285],
       [231, 293],
       [167, 105],
       [ 33,  95],
       [251, 267],
       [147, 131],
       [  9,  78],
       [191, 122],
       [ 53,  69],
       [207, 276],
       [285, 223],
       [113, 175],
       [ 87,  25],
       [ 97,   8],
       [103, 192],
       [295, 206],
       [294, 210],
       [104, 188],
       [ 96,  12],
       [ 68,  45],
       [132, 155],
       [266, 243],
       [ 99,   2],
       [101, 198],
       [297, 200]])

#### histogram of kde 2d

In [21]:
plt.figure()
plt.hist(z_kde_grid.reshape(-1), bins=100)
plt.show()

#### show entire kde image

In [22]:
plt.figure()
plt.title('z_kde_grid')
plt.imshow(z_kde_grid)
plt.plot(pk_coords_mir[:, 1], pk_coords_mir[:, 0], 'r.')  # note columns' order
plt.gca().invert_yaxis()
plt.show()

### get peak index

In [23]:
pk_coords_mir_correct = np.vstack(
    [pk_coords_mir[:, 1], pk_coords_mir[:, 0]]).T  # make columns order correct to calculate distance later
pk_coords_mir_correct

array([[ 33,  79],
       [167, 121],
       [231, 277],
       [238, 271],
       [160, 127],
       [ 40,  73],
       [290, 238],
       [108, 160],
       [ 92,  40],
       [284, 242],
       [114, 156],
       [ 86,  44],
       [ 99,  28],
       [101, 172],
       [297, 226],
       [247, 262],
       [151, 136],
       [ 49,  64],
       [ 61,  55],
       [139, 145],
       [259, 253],
       [ 87,  16],
       [113, 184],
       [285, 214],
       [293, 231],
       [105, 167],
       [ 95,  33],
       [267, 251],
       [131, 147],
       [ 78,   9],
       [122, 191],
       [ 69,  53],
       [276, 207],
       [223, 285],
       [175, 113],
       [ 25,  87],
       [  8,  97],
       [192, 103],
       [206, 295],
       [210, 294],
       [188, 104],
       [ 12,  96],
       [ 45,  68],
       [155, 132],
       [243, 266],
       [  2,  99],
       [198, 101],
       [200, 297]])

find the close one to pk_coord in data

In [24]:
pk_coords_unmir_correct = []
for pk in pk_coords_mir_correct:
    x, y = pk[0], pk[1]
    if x in range(grid_size, 2 * grid_size) and y in range(grid_size, 2 * grid_size):
        pk_coords_unmir_correct.append(pk)
print(pk_coords_unmir_correct)
pk_coords_unmir_correct = np.array(pk_coords_unmir_correct) - grid_size
pk_coords_unmir_correct

[array([167, 121]), array([160, 127]), array([108, 160]), array([114, 156]), array([101, 172]), array([151, 136]), array([139, 145]), array([113, 184]), array([105, 167]), array([131, 147]), array([122, 191]), array([175, 113]), array([192, 103]), array([188, 104]), array([155, 132]), array([198, 101])]


array([[67, 21],
       [60, 27],
       [ 8, 60],
       [14, 56],
       [ 1, 72],
       [51, 36],
       [39, 45],
       [13, 84],
       [ 5, 67],
       [31, 47],
       [22, 91],
       [75, 13],
       [92,  3],
       [88,  4],
       [55, 32],
       [98,  1]])

In [25]:
from scipy.spatial import distance

pk_point_idx = []
print('pk_coord, min_dist_point, min_dist, min_dist_point_idx')

for pk_coord in pk_coords_unmir_correct:
    min_dist = math.inf
    min_dist_point_idx = -1  # idx
    for i, point in enumerate(data):
        # if i not in range(grid_size, 2*grid_size):
        #     continue
        dist = distance.euclidean(point, pk_coord)
        if dist < min_dist:
            min_dist = dist
            min_dist_point = point
            min_dist_point_idx = i
    print(pk_coord, min_dist_point, min_dist, min_dist_point_idx)
    pk_point_idx.append(min_dist_point_idx)
print(pk_point_idx)

pk_coord, min_dist_point, min_dist, min_dist_point_idx
[67 21] [67.49688217 21.11389997] 0.5097696496705368 96
[60 27] [60.56271394 27.55088785] 0.7874797788232187 88
[ 8 60] [ 8.4000544  60.49706055] 0.63805384480595 44
[14 56] [14.37948685 56.15913577] 0.4115026914141611 50
[ 1 72] [ 0.26490699 71.78317841] 0.7664028574489219 32
[51 36] [51.69028308 36.42504392] 0.8106497765989114 78
[39 45] [38.77957598 45.288051  ] 0.362712186650839 67
[13 84] [12.91490279 84.3352878 ] 0.3459182606933599 17
[ 5 67] [ 5.0937085  67.63798566] 0.6448309731786107 37
[31 47] [31.62332571 47.82410585] 1.0332886280174434 62
[22 91] [21.56988134 91.35662793] 0.5587356654365296 9
[75 13] [75.76079377 13.19855664] 0.7862772377803798 106
[92  3] [91.9318389  3.1698989] 0.18306166486442235 120
[88  4] [88.43321479  4.41587073] 0.6005193737151374 117
[55 32] [55.10571684 32.13884074] 0.1745073087910646 83
[98  1] [97.59862034  1.08673523] 0.41064416366575823 125
[96, 88, 44, 50, 32, 78, 67, 17, 37, 62, 9, 106, 

In [26]:
plt.figure()
# plt.scatter(x, y)
# plt.plot(x_s, y_s, color='red', marker='o', markerfacecolor='white', markeredgecolor='#1f77b4', markeredgewidth=1.5)
plt.plot(x_ori, y_ori, color='red', marker='o', markerfacecolor='white', markeredgecolor='#1f77b4', markeredgewidth=1.5)
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.ticklabel_format(style='plain', axis='x', useOffset=False)
# plt.scatter(x_s[pk_point_idx], y_s[pk_point_idx], c='red', zorder=100)
plt.scatter(x_ori[pk_point_idx], y_ori[pk_point_idx], c='red', zorder=100)
plt.show()